# We have aligned R IR G images, no AF (UV). R + G are aligned - we need to crop by largest intensity values present in columns / rows (crop should be done across all 3 images.

Resulting frames should be saved individually into folder C43/TIMESTAMP (OR OTHER UNIQUE ID)/:
g-g/g-g.png
g-r/g-r.png
g-b/g-b.png
...
...


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import sys
import os

# sys.path.append(os.path.abspath("/mnt/skin_cancer_nas/eye_vein_pulse/pipelines"))
# sys.path.append(os.path.abspath("/mnt/skin_cancer_nas/eye_vein_pulse/pipelines/frames_from_video"))
# sys.path.append(os.path.abspath("/mnt/skin_cancer_nas/eye_vein_pulse/data/torch_generator"))
# sys.path.append(os.path.abspath("/mnt/skin_cancer_nas/eye_vein_pulse/nas/darts_torch"))
# sys.path.append('/mnt')
# sys.path.append('/mnt/src')
# sys.path.append('/mnt/src/eye_vein_pulse')

# from src.eye_vein_pulse.pipelines.frames_from_video import nodes as frames_from_video_nodes

import PIL
from PIL import Image
from numpy import *
from pylab import *

import cv2

from tqdm.notebook import tqdm

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML
from matplotlib.pyplot import imshow

from sklearn.decomposition import PCA



import cv2

# from skimage.filters import meijering

In [3]:
%matplotlib inline

In [5]:
os.listdir('/mnt/data/raw/Usable/57.2')

['mel_images_imgR.png',
 'aligned_images_imgR.png',
 'resulting_parameter_map.mat',
 'aligned_images_imgIR.png',
 'marker_binary_images.png',
 'malformation_MaskGRP.png',
 'mel_images_imgIR.png',
 'result_parameter_map_jet.png',
 'aligned_images.png',
 'aligned_images_imgG.png',
 'result_parameter_map_3col.png',
 'mel_images_imgG.png']

In [11]:
mel_images_imgR = cv2.imread('/mnt/data/raw/Usable/57.2/mel_images_imgG.png', cv2.IMREAD_COLOR)

In [277]:
from scipy.optimize import minimize

def inscribe_rectangle(img):
    '''
    looking for a rectangle inscribed, so that it does not 
    have any zero values potentially present on borders or inside.
    
    returns array [y_min, x_min, y_max, x_max]
    '''
    img_center_x = img.shape[1] // 2
    img_center_y = img.shape[0] // 2
    delta = 20
    
    def f(_x):
        '''
        we are minimizing count of zeros, but increasing area
        '''
        _img = img[int(_x[0]):int(_x[2]), int(_x[1]):int(_x[3])]
        zero_count = np.count_nonzero(_img==0)
        area = _img.shape[0] * _img.shape[1]
        return area*(-1) + zero_count * 100000
    
    x0 = [img_center_y-delta, img_center_x-delta, img_center_y+delta, img_center_x+delta]
    res = minimize(f, x0, method='Nelder-Mead', tol=1e-1)
    return res.x.astype('uint16')
    

def collect_channels_dict_from_folder(images_folder: str):
    img_list = os.listdir(images_folder)
    d = {}
    for img_file_name in img_list:
        if img_file_name.lower().startswith('mel'):
            img_path = os.path.join(images_folder, img_file_name)
#             img_name_only = os.path.basename(img_path)
#             print('img_name_only',img_name_only)
            channel = 'nan'
            if img_path.endswith('_imgR.png'):
                channel = 'r'
            elif img_path.endswith('_imgIR.png'):
                channel = 'ir'
            elif img_path.endswith('_imgG.png'):
                channel = 'g'
            
            img3ch = cv2.imread(img_path, cv2.IMREAD_COLOR)  # CV2 reads images in BGR order
            d['path-{}'.format(channel)] = img_path
            d['{}-b'.format(channel)] = img3ch[:, :, 0]
            d['{}-g'.format(channel)] = img3ch[:, :, 1]
            d['{}-r'.format(channel)] = img3ch[:, :, 2]
    
    del d['r-b']
    del d['r-g']
    del d['ir-b']
    del d['ir-g']
    del d['g-b']
    del d['g-r']
    
# #     res_r = inscribe_rectangle(d['r-r'])  # ir & g channels are aligned to R
#     res_ir = inscribe_rectangle(d['ir-r'])
#     print('res_ir', res_ir)
#     res_g = inscribe_rectangle(d['g-g'])
#     print('res_g', res_g)
    
#     res_all = np.vstack((res_ir, res_g))
#     y_min = res_all[:, 0].max()
#     x_min = res_all[:, 1].max()
#     y_max = res_all[:, 2].min()
#     x_max = res_all[:, 3].min()
#     print('res_all', res_all)
#     print(y_min, x_min, y_max, x_max)
    
#     crop_range = 1
    
#     while True:
#         y_min = crop_range
#         x_min = crop_range
#         y_max = d['r-r'].shape[0] - crop_range
#         x_max = d['r-r'].shape[1] - crop_range

#         d['r-r'] = d['r-r'][y_min:y_max, x_min:x_max]
#         d['ir-r'] = d['ir-r'][y_min:y_max, x_min:x_max]
#         d['g-g'] = d['g-g'][y_min:y_max, x_min:x_max]

#         zero_count_ir = np.count_nonzero(d['ir-r']==0)
#         zero_count_g = np.count_nonzero(d['g-g']==0)
        
# #         print(zero_count_ir, zero_count_g)
        
#         if zero_count_ir + zero_count_g == 0:
#             break
    
#     area_sum_ir_r = d['r-r'].shape[0] * d['r-r'].shape[1]
    
#     if area_sum_ir_r == 0:
#         print('images_folder', images_folder, 'cropped everything!')
#         return None
#     else:
#         return d
    
    crop_range = 45
    y_min = crop_range
    x_min = crop_range
    y_max = d['r-r'].shape[0] - crop_range
    x_max = d['r-r'].shape[1] - crop_range

    d['r-r'] = d['r-r'][y_min:y_max, x_min:x_max]
    d['ir-r'] = d['ir-r'][y_min:y_max, x_min:x_max]
    d['g-g'] = d['g-g'][y_min:y_max, x_min:x_max]
    return d

def process_d22_converted_to_c43_root_folder(root='/mnt/data/raw/Usable'):
    '''
    Function processes root folder with D22 images augmented (Katrina Bolonko matlab code) 
    so that P-value based analytic algorithm detects them as C43 (Melanoma).
    
    This function creates interim dataset with our standard greyscale images in folders like g-g/g-r/g-b/r-r/r-b/r-g/ir-r/ir-g/ir-b. 
    These images will be slightly cropped to remove saturated colors thy posses on borders (thanks to registration (made by K.Bolonko)).
    '''
    dicts_list = []
    subdirs_list = os.listdir(root)
    for subdir in tqdm(subdirs_list, desc='loading images per channels from root folder'):
        images_folder = os.path.join(root, subdir)       
        d = collect_channels_dict_from_folder(images_folder)
        if d is not None:
            dicts_list.append(d)
    return dicts_list

In [278]:
d_list = process_d22_converted_to_c43_root_folder()

# Create output folder with our structure folder_name/ir-r/ir-r.png, folder_name/g-g/g-g.png, ... 

In [279]:
def save_images(d_list, out_folder):
    for d in tqdm(d_list, desc='Saving images into new folders structure'):
        folder_name = os.path.basename(os.path.dirname(d['path-r']))
        new_out_folder = os.path.join(out_folder, folder_name)
        
        ir_r = d['ir-r']
        r_r = d['r-r']
        g_g = d['g-g']
        
        print(ir_r.shape, r_r.shape, g_g.shape)
        
        ir_r_file_path = os.path.join(new_out_folder, 'ir-r', 'ir-r.png')
        r_r_file_path = os.path.join(new_out_folder, 'r-r', 'r-r.png')
        g_g_file_path = os.path.join(new_out_folder, 'g-g', 'g-g.png')

        print('ir_r_file_path=',ir_r_file_path)
        
        os.makedirs(os.path.dirname(ir_r_file_path), exist_ok = True)
        os.makedirs(os.path.dirname(r_r_file_path), exist_ok = True)
        os.makedirs(os.path.dirname(g_g_file_path), exist_ok = True)
        
        cv2.imwrite(ir_r_file_path, ir_r)
        cv2.imwrite(r_r_file_path, r_r)
        cv2.imwrite(g_g_file_path, g_g)
    print('Done.')

In [280]:
save_images(d_list, '/mnt/data/interim/_melanoma_artificial2/C43')

(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1540/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1530/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/657/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/29.2/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/881/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1863/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/714/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/137.4/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/981/ir-r/ir-r

(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1728/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1710/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/20190318_12-17-32/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/20190306_11-05-35/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1348/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1364/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/94.4/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/744/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_ar

(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1106/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/217.4/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/20190320_12-38-26/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/122.9/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1352/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/698/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/51.2/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/28.2/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C

(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1804/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/218.1/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/140.4/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/1565/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/851/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/51.1/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/804/ir-r/ir-r.png
(390, 550) (390, 550) (390, 550)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/147.7/ir-r/ir-r.png
(358, 376) (358, 376) (358, 376)
ir_r_file_path= /mnt/data/interim/_melanoma_artificial2/C43/20190326_1